In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap

import io
from pysam import VariantFile

from sklearn.metrics import precision_score, recall_score, average_precision_score, precision_recall_curve

In [2]:
tab20 = cm.get_cmap('tab10', 8)
newcmap_list = []
for i in range(4):
    newcmap_list.append(tab20.colors[i])
for i in range(4):
    newcmap_list.append(np.array(list(tab20.colors[i][:-1]) +[0.3]))

newcmap = ListedColormap(newcmap_list, name='newcmap')
color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

In [3]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

foo = lambda x: pd.Series(x.split('CALLERS=')[1].split(';')[0])
foo2 = lambda x: pd.Series(x.split('TYPE=')[1].split(';')[0] if len(x.split('TYPE=')) > 1 else np.nan)

In [4]:
methods = ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']

In [5]:
sns.set(style="darkgrid", context="talk", rc={"lines.linewidth": 2})
plt.style.use("dark_background")
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set_palette("deep")

# Select plasma sample

In [6]:
#sample = '809'
sample = '986'

In [7]:
if sample == '809':
    plasmasample1 = '809_110914'
    plasmasample2 = '809_030915'
    tumorsample1 = '809_290714-T1W'
elif sample == '986':
    plasmasample1 = '986_100215'
    plasmasample2 = '986_261016'
    tumorsample1 = '986_100215-T1W'

# Load SNV calls for plasma sample and matching mixed samples

In [9]:
vcf_pd_0 = read_vcf("data/calls_dilution/2015-07-31_CRC-"+plasmasample1+"-1-0_filtered/CRC-"+plasmasample1+"-1-0_filtered-ensemble-annotated.vcf")
vcf_pd_0.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/calls_dilution/2015-07-31_CRC-986_100215-1-0_filtered/CRC-986_100215-1-0_filtered-ensemble-annotated.vcf'

In [ ]:
vcf_pd_0 = read_vcf("../data/calls_dilution/2015-07-31_CRC-"+plasmasample1+"-1-0_filtered/CRC-"+plasmasample1+"-1-0_filtered-ensemble-annotated.vcf")
vcf_pd_1 = read_vcf("../data/calls_dilution/2015-07-31_CRC-MSS-SNV-tf0.1/CRC-MSS-SNV-tf0_1-ensemble-annotated.vcf")

vcf_pd_0['callers'] = vcf_pd_0['INFO'].apply(foo)
vcf_pd_0['type'] = np.nan
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() == 0] = 'SNV'
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() > 0] = 'INS'
vcf_pd_0['type'][vcf_pd_0['ALT'].str.len() - vcf_pd_0['REF'].str.len() < 0] = 'DEL'
vcf_pd_0['type'][vcf_pd_0['ID'].str.contains('rs')] = 'SNP'
vcf_pd_0['freebayes'] = vcf_pd_0['INFO'].str.contains('freebayes')
vcf_pd_0['vardict'] = vcf_pd_0['INFO'].str.contains('vardict')
vcf_pd_0['varscan'] = vcf_pd_0['INFO'].str.contains('varscan')
vcf_pd_0['mutect2'] = vcf_pd_0['INFO'].str.contains('mutect2')
vcf_pd_0['strelka2'] = vcf_pd_0['INFO'].str.contains('strelka2')

vcf_pd_1['callers'] = vcf_pd_1['INFO'].apply(foo)
vcf_pd_1['type'] = np.nan
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() == 0] = 'SNV'
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() > 0] = 'INS'
vcf_pd_1['type'][vcf_pd_1['ALT'].str.len() - vcf_pd_1['REF'].str.len() < 0] = 'DEL'
vcf_pd_1['type'][vcf_pd_1['ID'].str.contains('rs')] = 'SNP'
vcf_pd_1['freebayes'] = vcf_pd_1['INFO'].str.contains('freebayes')
vcf_pd_1['vardict'] = vcf_pd_1['INFO'].str.contains('vardict')
vcf_pd_1['varscan'] = vcf_pd_1['INFO'].str.contains('varscan')
vcf_pd_1['mutect2'] = vcf_pd_1['INFO'].str.contains('mutect2')
vcf_pd_1['strelka2'] = vcf_pd_1['INFO'].str.contains('strelka2')


sample_0 = vcf_pd_0[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_0['CHROM_POS'] = sample_0['CHROM'].astype('str').str.cat(sample_0['POS'].astype('str'),sep="_")
sample_0.set_index('CHROM_POS', inplace = True)

sample_1 = vcf_pd_1[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_1['CHROM_POS'] = sample_1['CHROM'].astype('str').str.cat(sample_1['POS'].astype('str'),sep="_")
sample_1.set_index('CHROM_POS', inplace = True)

sample_1.head()

In [ ]:
vcf_pd_0_bis = read_vcf("../data/calls_dilution/2015-07-31_CRC-"+plasmasample2+"-1-0_filtered/CRC-"+plasmasample2+"-1-0_filtered-ensemble-annotated.vcf")

vcf_pd_0_bis['type'] = np.nan
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() == 0] = 'SNV'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() > 0] = 'INS'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ALT'].str.len() - vcf_pd_0_bis['REF'].str.len() < 0] = 'DEL'
vcf_pd_0_bis['type'][vcf_pd_0_bis['ID'].str.contains('rs')] = 'SNP'
vcf_pd_0_bis['freebayes'] = vcf_pd_0_bis['INFO'].str.contains('freebayes')
vcf_pd_0_bis['vardict'] = vcf_pd_0_bis['INFO'].str.contains('vardict')
vcf_pd_0_bis['varscan'] = vcf_pd_0_bis['INFO'].str.contains('varscan')
vcf_pd_0_bis['mutect2'] = vcf_pd_0_bis['INFO'].str.contains('mutect2')
vcf_pd_0_bis['strelka2'] = vcf_pd_0_bis['INFO'].str.contains('strelka2')

sample_0_bis = vcf_pd_0_bis[['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'type', 'freebayes', 'vardict', 'varscan', 'mutect2', 'strelka2']]
sample_0_bis['CHROM_POS'] = sample_0_bis['CHROM'].astype('str').str.cat(sample_0_bis['POS'].astype('str'),sep="_")
sample_0_bis.set_index('CHROM_POS', inplace = True)
sample_0_bis.head()

sample0 = sample_0.copy()
sample0['date'] = '110914'
sample0bis = sample_0_bis.copy()
sample0bis['date'] = '030915'
sample_ref = pd.concat([sample0, sample0bis], join='inner')
l1 = list(sample_ref[sample_ref['date'] == '110914'].index)
l2 = list(sample_ref[sample_ref['date'] == '030915'].index)
print(len(l1), len(l2))
print(len(list(set(set(l1) ^ set(l2)))))
print(len(list(set(set(l1) & set(l2)))))

# Estimated tumor burden of mixed samples

In [ ]:
samples_tf = {
    'sample_0': float(list(pd.read_csv("../data/calls_dilution/2015-07-31_CRC-"+plasmasample1+"-1-0_filtered/estimated_tf.txt").columns)[0]),
    'sample_1': 0.1,

}

print(samples_tf)

# Number of detections detected

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[s[method] == True].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & ((s['type'] == 'INS') | (s['type'] == 'DEL'))].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & (s['type'] == 'SNV')].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

In [ ]:
numbersnvs_pd = pd.DataFrame()

for si, s in enumerate([sample_0, sample_1]):
    nb_snv = []
    for method in methods:
        #if si == 4:
        #    si = 't'
       # print('sample '+ str(si) + ': ', method, s[s[method] == True].shape[0])
        nb_snv.append(s[(s[method] == True) & (s['type'] == 'SNP')].shape[0])
    if si == 0:
        numbersnvs_pd = pd.DataFrame.from_dict({'sample_'+ str(si): nb_snv}).T
        numbersnvs_pd.columns = methods
    else:
        numbersnvs_pd.loc['sample_'+ str(si)] = nb_snv
    numbersnvs_pd = numbersnvs_pd.rename(index=samples_tf)

numbersnvs_pd.plot(style='.-', logx=True, xlim=(10e-1, 10e-3))
#sns.catplot(x='tumor burden', y='coverage', kind='point', data=coverage_pd,
#            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True))
        
numbersnvs_pd

# Coverage variability

In [ ]:
samples_cov_tf = {
    'sample_0': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-0/estimated_tf.txt").columns)[0]),2)],
    'sample_1': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-1-05775/estimated_tf.txt").columns)[0]),2)],
    'sample_2': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-075-06738/estimated_tf.txt").columns)[0]),2)],
    'sample_3': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-05-07701/estimated_tf.txt").columns)[0]),2)],
    'sample_4': [float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/coverage.txt").columns)[0]),
                np.round(float(list(pd.read_csv("../data/2015-07-31_CRC-"+plasmasample1+"-025-08663/estimated_tf.txt").columns)[0]),2)],
    
}

print(samples_cov_tf)

coverage_pd = pd.DataFrame.from_dict(samples_cov_tf).T
coverage_pd.columns = ['coverage', 'tumor burden']
sns.catplot(x='tumor burden', y='coverage', data=coverage_pd,
            order=sorted(coverage_pd['tumor burden'].unique(), reverse=True), color='magenta')

        
coverage_pd

In [ ]:
22	28456078	28456078	0.03	A
22	46660419	46660419	0.0363	A
22	50527428	50527428	0.0417	A

In [ ]:
#sample_1.loc['22_46660419']
#sample_1.loc['22_28456078']
#sample_1.loc['22_50527428']
poss1 = [s for s in list(sample_1.index) if s.startswith('22_')]
poss1

In [ ]:
print('22_46660419' in poss1)
#sample_1.loc['22_28456078']

# AUPRC with reference = SNV found in undiluted plasma sample by the same method

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(methods):
        #print(method)

        s0 = sample_0[['QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s1 = sample_1[['QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)

        pd_method = pd.concat([s0, s1], axis=1)

        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0', 'QUAL_1', 'type', 'sample_1']
        pd_method[['sample_0', 'sample_1']] = pd_method[['sample_0','sample_1']].fillna(value=False)
        pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_1'] == False].index, axis=0, inplace=True)
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_1']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]
                                
        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')
    #print(pd_methods.shape)

    pd_results = pd.DataFrame
    pd_results_PR = pd.DataFrame
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')
    
    y_true = pd_methods[['sample_0', 'caller']]
    y_true.index.name = 'CHROM_POS'
    y_true = y_true.groupby(['CHROM_POS'])['sample_0'].sum()
    y_true[y_true <= 3] = 0
    y_true = y_true.astype(bool)
    print(y_true.shape[0])
    print(pd_methods.shape)


    baselineAUPRC = {}


    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

       # y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)
        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        if method == 'mutect2':
            print(y_0[y_0 == True])
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        

        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method

        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 2], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0.4, 1, .8, .6, .4, .2, 0.1]
        for i in range(1,3):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    plt.figure()
    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'w', linestyle = '--') 
    #plt.ylim([0.1, 0.5])
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'w', linestyle = '--', label='baseline AUPRC') 

    plt.figure()
    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(methods):
        #print(method)

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s1 = sample_1[['QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)

        pd_method = pd.concat([s0, s1], axis=1)


        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0', 'QUAL_1', 'type_1', 'sample_1']
        pd_method[['sample_0', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']] = pd_method[['sample_0','sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']].fillna(value=False)
        pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_1'] + pd_method['sample_2']  + pd_method['sample_3'] + pd_method['sample_4'] + pd_method['sample_5'] == False].index, axis=0, inplace=True)
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_1', 'type_2', 'type_3', 'type_4', 'type_5']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]
                                
        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')
    #print(pd_methods.shape)

    pd_results = pd.DataFrame
    pd_results_PR = pd.DataFrame
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')
    
    y_true = pd_methods[['sample_0', 'caller']]
    y_true.index.name = 'CHROM_POS'
    y_true = y_true.groupby(['CHROM_POS'])['sample_0'].sum()
    y_true[y_true <= 3] = 0
    y_true = y_true.astype(bool)
    print(y_true.shape[0])
    print(pd_methods.shape)


    baselineAUPRC = {}


    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

       # y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)
        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        if method == 'mutect2':
            print(y_0[y_0 == True])
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)
        y_5 = pd_methods[pd_methods['caller'] == method]['sample_5'].astype(bool).reindex(y_true.index).squeeze()
        y_5 = y_5.fillna(False)


        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                          average_precision_score(y_true, y_5),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                      average_precision_score(y_true, y_5) - baselineAUPRC[method],

                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                          precision_score(y_true, y_5),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                          recall_score(y_true, y_5),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method

        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 5], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0.4, 1, .8, .6, .4, .2, 0.1]
        for i in range(1,5):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    plt.figure()
    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'w', linestyle = '--') 
    #plt.ylim([0.1, 0.5])
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'w', linestyle = '--', label='baseline AUPRC') 

    plt.figure()
    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

# AUPRC with reference = SNV found in both undiluted plasma samples by the same method

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']):

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        s0_bis = sample_0_bis[['REF', 'ALT', 'QUAL', 'type', method]]
        s0_bis.rename(columns = {method:'sample_0_bis'},  inplace = True)
        s1 = sample_1[['REF', 'ALT', 'QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)
        s2 = sample_2[['REF', 'ALT', 'QUAL', 'type', method]]
        s2.rename(columns = {method:'sample_2'},  inplace = True)
        s3 = sample_3[['REF', 'ALT', 'QUAL', 'type', method]]
        s3.rename(columns = {method:'sample_3'},  inplace = True)
        s4 = sample_4[['REF', 'ALT', 'QUAL','type', method]]
        s4.rename(columns = {method:'sample_4'},  inplace = True)
        s5 = sample_5[['REF', 'ALT', 'QUAL','type', method]]
        s5.rename(columns = {method:'sample_5'},  inplace = True)

        pd_method = pd.concat([s0, s0_bis, s1, s2, s3, s4, s5], axis=1)

        pd_method['REF'] = [list(np.unique([i for i in list(ai) if str(i) != 'nan']))[-1] for ai in list(pd_method['REF'].values)]
        pd_method['ALT'] = [list(np.unique([i for i in list(ai) if str(i) != 'nan']))[-1] for ai in list(pd_method['ALT'].values)]
        pd_method = pd_method.T.drop_duplicates().T
        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0','QUAL_0_bis', 'type_0_bis', 'sample_0_bis', 'QUAL_1', 'type_1', 'sample_1', 'QUAL_2', 'type_2', 'sample_2',  'QUAL_3', 'type_3', 'sample_3', 'QUAL_4', 'type_4', 'sample_4', 'QUAL_5', 'type_5', 'sample_5']
        pd_method[['sample_0', 'sample_0_bis', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']] = pd_method[['sample_0', 'sample_0_bis', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']].fillna(value=False)
        #pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_0_bis']+ pd_method['sample_1'] + pd_method['sample_2'] + pd_method['sample_3'] + pd_method['sample_4'] == False].index, axis=0, inplace=True)
        pd_method[['REF', 'sample_0', 'sample_0_bis', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']].head()
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_0_bis','type_1', 'type_2', 'type_3', 'type_4', 'type_5']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]

        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')

    pd_results = pd.DataFrame.empty
    pd_results_PR = pd.DataFrame.empty
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')


    y_true = pd_methods[['sample_0', 'sample_0_bis', 'caller']]
    y_true['ref'] = pd_methods['sample_0'] & pd_methods['sample_0_bis']
    y_true.index.name = 'CHROM_POS'
    y_true = y_true.groupby(['CHROM_POS'])['ref'].sum()
    y_true = y_true.astype(bool)


    baselineAUPRC = {}

    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

        #y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool) + pd_methods[pd_methods['caller'] == method]['sample_0_bis'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)

        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)
        y_5 = pd_methods[pd_methods['caller'] == method]['sample_5'].astype(bool).reindex(y_true.index).squeeze()
        y_5 = y_5.fillna(False)

        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                          average_precision_score(y_true, y_5),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                      average_precision_score(y_true, y_5) - baselineAUPRC[method],
                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                          precision_score(y_true, y_5),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                          recall_score(y_true, y_5),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method
        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 5], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [1, 1, .8, .6, .4, .2, .1]
        for i in range(1,6):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results, legend=False)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'w', linestyle = '--') 
    #plt.ylim([0, 0.5])
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'w', linestyle = '--', label='baseline AUPRC') 

    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

## AUPRC with reference = SNV found in the tumor by the same method

In [ ]:
for mutation in ['snv', 'indel']:
    pd_methods = pd.DataFrame
    count = 0

    for mi, method in enumerate(['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']):

        s0 = sample_0[['REF', 'ALT', 'QUAL', 'type', method]]
        s0.rename(columns = {method:'sample_0'},  inplace = True)
        st = sample_t[['REF', 'ALT', 'QUAL', 'type', method]]
        st.rename(columns = {method:'sample_t'},  inplace = True)
        s1 = sample_1[['REF', 'ALT', 'QUAL', 'type', method]]
        s1.rename(columns = {method:'sample_1'},  inplace = True)
        s2 = sample_2[['REF', 'ALT', 'QUAL', 'type', method]]
        s2.rename(columns = {method:'sample_2'},  inplace = True)
        s3 = sample_3[['REF', 'ALT', 'QUAL', 'type', method]]
        s3.rename(columns = {method:'sample_3'},  inplace = True)
        s4 = sample_4[['REF', 'ALT', 'QUAL','type', method]]
        s4.rename(columns = {method:'sample_4'},  inplace = True)
        s5 = sample_5[['REF', 'ALT', 'QUAL','type', method]]
        s5.rename(columns = {method:'sample_5'},  inplace = True)

        pd_method = pd.concat([s0, st, s1, s2, s3, s4, s5], axis=1)

        pd_method['REF'] = [list(np.unique([i for i in list(ai) if str(i) != 'nan']))[-1] for ai in list(pd_method['REF'].values)]
        pd_method['ALT'] = [list(np.unique([i for i in list(ai) if str(i) != 'nan']))[-1] for ai in list(pd_method['ALT'].values)]
        pd_method = pd_method.T.drop_duplicates().T
        pd_method.columns = ['REF', 'ALT', 'QUAL_0', 'type_0', 'sample_0','QUAL_t', 'type_t', 'sample_t', 'QUAL_1', 'type_1', 'sample_1', 'QUAL_2', 'type_2', 'sample_2',  'QUAL_3', 'type_3', 'sample_3', 'QUAL_4', 'type_4', 'sample_4', 'QUAL_5', 'type_5', 'sample_5']
        pd_method[['sample_0', 'sample_t', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']] = pd_method[['sample_0', 'sample_t', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']].fillna(value=False)
        #pd_method.drop(pd_method[pd_method['sample_0'] + pd_method['sample_0_bis']+ pd_method['sample_1'] + pd_method['sample_2'] + pd_method['sample_3'] + pd_method['sample_4'] == False].index, axis=0, inplace=True)
        pd_method[['REF', 'sample_0', 'sample_t', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']].head()
        pd_method['caller'] = method
        pd_method['type'] = pd_method[['type_0','type_t','type_1', 'type_2', 'type_3', 'type_4', 'type_5']].fillna('').sum(axis=1).apply(lambda x: pd.Series(x[:3]))

        if mutation == 'snv':
            pd_method = pd_method[pd_method['type'] == 'SNV']
        elif mutation == 'indel':
            pd_method = pd_method[(pd_method['type'] == 'INS') | (pd_method['type'] == 'DEL')]

        if count == 0:
            pd_methods = pd_method
            count = 1
        else:
            pd_methods = pd.concat([pd_methods, pd_method], join='inner')

    pd_results = pd.DataFrame.empty
    pd_results_PR = pd.DataFrame.empty
    count = 0

    fig, axs = plt.subplots(1,5,figsize=(30, 4))
    fig.suptitle('Precision-Recall curves')


    y_true = pd_methods[['sample_t', 'caller']]
    y_true.index.name = 'CHROM_POS'
    y_true = y_true.groupby(['CHROM_POS'])['sample_t'].sum()
    y_true[y_true <= 2] = 0
    y_true = y_true.astype(bool)
    print(y_true.shape[0])



    baselineAUPRC = {}

    for mi, method in enumerate(methods):

        res_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')
        res_PR_df = (100*pd.Series(samples_tf)).round(decimals=2).to_frame(name='tumor burden')

        #y_true = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool) + pd_methods[pd_methods['caller'] == method]['sample_0_bis'].astype(bool)
        baselineAUPRC[method] = len(y_true[y_true == True])/len(y_true)

        y_0 = pd_methods[pd_methods['caller'] == method]['sample_0'].astype(bool).reindex(y_true.index).squeeze()
        y_0 = y_0.fillna(False)
        y_1 = pd_methods[pd_methods['caller'] == method]['sample_1'].astype(bool).reindex(y_true.index).squeeze()
        y_1 = y_1.fillna(False)
        y_2 = pd_methods[pd_methods['caller'] == method]['sample_2'].astype(bool).reindex(y_true.index).squeeze()
        y_2 = y_2.fillna(False)
        y_3 = pd_methods[pd_methods['caller'] == method]['sample_3'].astype(bool).reindex(y_true.index).squeeze()
        y_3 = y_3.fillna(False)
        y_4 = pd_methods[pd_methods['caller'] == method]['sample_4'].astype(bool).reindex(y_true.index).squeeze()
        y_4 = y_4.fillna(False)
        y_5 = pd_methods[pd_methods['caller'] == method]['sample_5'].astype(bool).reindex(y_true.index).squeeze()
        y_5 = y_5.fillna(False)

        res_df['AUPRC'] = [average_precision_score(y_true, y_0),
                          average_precision_score(y_true, y_1),
                          average_precision_score(y_true, y_2),
                          average_precision_score(y_true, y_3),
                          average_precision_score(y_true, y_4),
                          average_precision_score(y_true, y_5),
                         ]
        res_df['AUPRC - baseline AUPRC'] = [average_precision_score(y_true, y_0) - baselineAUPRC[method],
                      average_precision_score(y_true, y_1) - baselineAUPRC[method],
                      average_precision_score(y_true, y_2) - baselineAUPRC[method],
                      average_precision_score(y_true, y_3) - baselineAUPRC[method],
                      average_precision_score(y_true, y_4) - baselineAUPRC[method],
                      average_precision_score(y_true, y_5) - baselineAUPRC[method],

                     ]
        res_PR_df['precision'] = [precision_score(y_true, y_0),
                          precision_score(y_true, y_1),
                          precision_score(y_true, y_2),
                          precision_score(y_true, y_3),
                          precision_score(y_true, y_4),
                          precision_score(y_true, y_5),
                         ]

        res_PR_df['recall'] = [recall_score(y_true, y_0),
                          recall_score(y_true, y_1),
                          recall_score(y_true, y_2),
                          recall_score(y_true, y_3),
                          recall_score(y_true, y_4),
                          recall_score(y_true, y_5),
                         ]
        res_PR_df = pd.melt(res_PR_df, id_vars =['tumor burden'], value_vars =['precision', 'recall'],
                        var_name='metric', value_name='value')
        res_df['caller'] = method
        res_PR_df['caller'] = method
        res_df.drop(['sample_0'], inplace=True)
        res_PR_df.drop([0, 6], inplace=True)

        # Plot Precision-Recall curve
        alpha_list = [0, 1, .8, .6, .4, .2, .1]
        for i in range(1,6):
            y_i = pd_methods[pd_methods['caller'] == method]['sample_'+str(i)].astype(bool).reindex(y_true.index).squeeze()
            y_i = y_i.fillna(False)
            precision, recall, _ = precision_recall_curve(y_true, y_i)
            axs[mi].plot(recall, precision, 'o-',
                                  label='tf='+str(res_df['tumor burden'].loc['sample_'+str(i)])+ ', AP='+str(round(average_precision_score(y_true, y_i), 2)),
                                  c=color_list[mi], alpha=alpha_list[i])
        axs[mi].set_xlabel('Recall')
        axs[mi].set_ylabel('Precision')
        axs[mi].set_ylim([0.0, 1.05])
        axs[mi].set_xlim([0.0, 1.05])
        axs[mi].set_title(method)
        axs[mi].legend()

        if count == 0:
            pd_results = res_df
            pd_results_PR = res_PR_df
            count = 1
        else:
            pd_results = pd.concat([pd_results, res_df], join='inner')
            pd_results_PR = pd.concat([pd_results_PR, res_PR_df], join='inner')
    plt.show()

    sns.catplot(x="tumor burden", y="AUPRC", hue="caller",  
                    capsize=.2, height=4, aspect=1.5,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results, legend=False)
    for mi, method in enumerate(methods):
        plt.axhline(y = baselineAUPRC[method], color = 'w', linestyle = '--') 
    #plt.ylim([0, 0.5])
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    
    sns.catplot(x="tumor burden", y="AUPRC - baseline AUPRC", hue="caller",  
                capsize=.2, height=6, aspect=.75,
                kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results)
    plt.axhline(y = 0, color = 'w', linestyle = '--', label='baseline AUPRC') 

    sns.catplot(x="tumor burden", y="value", col="metric", hue="caller",  
                    capsize=.2, height=6, aspect=.75,
                    kind="point", order=sorted(pd_results['tumor burden'].unique(), reverse=True), data=pd_results_PR)
    #plt.ylim([0, 0.5])

- Histograms of true allele frequencies in each tumor sample. Note how increasing admixture increases the prevalence of low- frequency variants.
- Benchmarking results for germline SNVs
- Benchmarking results for somatic SNVs on exome data.
- averaged over the four replicates

- add fake mutations on healthy mixtures